In [1]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import os
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import datasets

In [3]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [4]:
# Freezing the base model
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the last classifier layer
for param in model.classifier.parameters():
    param.requires_grad = True

In [5]:
df = pd.read_csv('final_randomized_augmented.csv')

In [6]:
type(df.iloc[0]['Anger'])

numpy.float64

In [7]:
# # List of emotions in the GoEmotions dataset
# emotion_labels = [
#     "admiration", "amusement", "anger", "annoyance", "approval",
#     "caring", "confusion", "curiosity", "desire", "disappointment",
#     "disapproval", "disgust", "embarrassment", "excitement", "fear",
#     "gratitude", "grief", "joy", "love", "nervousness",
#     "optimism", "pride", "realization", "relief", "remorse",
#     "sadness", "surprise", "neutral"
# ]

# # Example: Finding the emotion name for label number 3
# label_number = 3  # Using 0-based index for label number
# emotion_name = emotion_labels[label_number]
# print("The emotion is:", emotion_name)

# Setting the labels of our dataset in the same sequence as above
df['labels'] = df.apply(lambda row:[float(row['Anger']), float(row['Fear']), float(row['Joy']), float(row['Sadness']), float(row['Surprise'])], axis=1)
df = df.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])
df

,text,labels
0,"She never listens, no matter how many times I ...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,But one criminal shoot from gun to my hand.,"[1.0, 1.0, 0.0, 0.0, 0.0]"
2,The fear of the unknown kept me frozen in plac...,"[0.0, 0.0, 1.0, 1.0, 0.0]"
3,What demon likes peas?,"[0.0, 0.0, 0.0, 0.0, 1.0]"
4,The last thing I expected was to find myself i...,"[0.0, 1.0, 0.0, 1.0, 0.0]"
...,...,...
4188,He keeps eating my leftovers without asking.,"[1.0, 0.0, 0.0, 0.0, 0.0]"
4189,"Pretty eerie, but awesome","[0.0, 1.0, 1.0, 0.0, 1.0]"
4190,They parked so close to my car that I couldn’t...,"[1.0, 1.0, 0.0, 0.0, 0.0]"
4191,It was one of those moments where time seemed ...,"[0.0, 1.0, 0.0, 0.0, 1.0]"


In [8]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)  # Use only the desired columns

# Optionally, print the dataset to check
print(dataset)

Dataset({
    features: ['text', 'labels'],
    num_rows: 4193
})


In [9]:
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to float, handling NoneType by treating it as 0, even in nested lists
    inputs["labels"] = [
        torch.tensor([l if l is not None else 0 for l in label], dtype=torch.float)
        if isinstance(label, list) else torch.tensor(label if label is not None else 0, dtype=torch.float)
        for label in examples["labels"]
    ]

    return inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

Map:   0%|          | 0/4193 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 4193
})

In [10]:
print(len(encoded_dataset[0]["labels"]))

5


In [11]:
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [12]:
print(encoded_dataset[0]["labels"])  # Should output a tensor of floats: e.g., tensor([1., 0., 0., ..., 0., 1.])


tensor([1., 1., 0., 0., 0.])


In [13]:
dataset = encoded_dataset.train_test_split(test_size=0.2)

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 3354
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 839
    })
})

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Sigmoid is typically used for multi-label classification to get probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # Convert probabilities to binary predictions (0 or 1)
    predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    accuracy = accuracy_score(labels, predictions)  # Multi-label accuracy

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [16]:
class ModifiedModel(torch.nn.Module):
    def __init__(self, original_model):
        super(ModifiedModel, self).__init__()
        self.original_model = original_model
        self.fc = torch.nn.Linear(11, 5)  # New FC layer to map 11 -> 5 labels

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.original_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: [batch_size, 11]
        logits = self.fc(logits)  # Shape: [batch_size, 5]
        if labels is not None:
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels.float())
            return {"loss": loss, "logits": logits}

        return {"logits": logits}

modified_model = ModifiedModel(model)

In [17]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [18]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.370500,0.303622,0.594756,0.792162,0.823984,0.763901
2,0.308200,0.290386,0.599523,0.807014,0.825950,0.790522
3,0.288000,0.291217,0.612634,0.800204,0.825950,0.779789
4,0.275000,0.285089,0.610250,0.802904,0.831125,0.778423
5,0.268100,0.280199,0.615018,0.809489,0.832198,0.788783


TrainOutput(global_step=2100, training_loss=0.3019519660586403, metrics={'train_runtime': 939.6301, 'train_samples_per_second': 17.847, 'train_steps_per_second': 2.235, 'total_flos': 0.0, 'train_loss': 0.3019519660586403, 'epoch': 5.0})

In [19]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.263300,0.281036,0.612634,0.814972,0.846375,0.787666
2,0.251400,0.277316,0.607867,0.813521,0.828229,0.800675
3,0.243600,0.284948,0.620977,0.806452,0.826933,0.789048
4,0.237900,0.281426,0.609058,0.806547,0.828592,0.787176


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.263300,0.281036,0.612634,0.814972,0.846375,0.787666
2,0.251400,0.277316,0.607867,0.813521,0.828229,0.800675
3,0.243600,0.284948,0.620977,0.806452,0.826933,0.789048
4,0.237900,0.281426,0.609058,0.806547,0.828592,0.787176
5,0.237500,0.275771,0.615018,0.811541,0.829782,0.795064


TrainOutput(global_step=2100, training_loss=0.24673085167294456, metrics={'train_runtime': 944.2113, 'train_samples_per_second': 17.761, 'train_steps_per_second': 2.224, 'total_flos': 0.0, 'train_loss': 0.24673085167294456, 'epoch': 5.0})

##Using the trained model to get predictions on the train and dev sets

In [20]:
df_val = pd.read_csv('eng_a.csv')
df_val

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,"My mouth fell open `` No, no, no... I..",NaN,NaN,NaN,NaN,NaN
1,eng_dev_track_a_00002,You can barely make out your daughter's pale f...,NaN,NaN,NaN,NaN,NaN
2,eng_dev_track_a_00003,But after blinking my eyes for a few times lep...,NaN,NaN,NaN,NaN,NaN
3,eng_dev_track_a_00004,Slowly rising to my feet I came to the conclus...,NaN,NaN,NaN,NaN,NaN
4,eng_dev_track_a_00005,I noticed this months after moving in and doin...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,"""ARcH stop your progression.",NaN,NaN,NaN,NaN,NaN
112,eng_dev_track_a_00113,"This 'star', starts to move across the sky.",NaN,NaN,NaN,NaN,NaN
113,eng_dev_track_a_00114,and my feet hurt.,NaN,NaN,NaN,NaN,NaN
114,eng_dev_track_a_00115,so i cried my eyes out and did the drawing.,NaN,NaN,NaN,NaN,NaN


In [21]:
df_val = df_val.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])
df_val

,text
0,"My mouth fell open `` No, no, no... I.."
1,You can barely make out your daughter's pale f...
2,But after blinking my eyes for a few times lep...
3,Slowly rising to my feet I came to the conclus...
4,I noticed this months after moving in and doin...
...,...
111,"""ARcH stop your progression."
112,"This 'star', starts to move across the sky."
113,and my feet hurt.
114,so i cried my eyes out and did the drawing.


In [22]:
val_dataset = Dataset.from_pandas(df_val)  # Use only the desired columns

# Optionally, print the dataset to check
print(val_dataset)

Dataset({
    features: ['text'],
    num_rows: 116
})


In [23]:
def preprocess_function_val(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to torch float to match the expected input type for BCEWithLogitsLoss
    # inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]

    return inputs

encoded_dataset_val = val_dataset.map(preprocess_function_val, batched=True)
encoded_dataset_val

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 116
})

In [24]:
encoded_dataset_val.set_format("torch", columns=["input_ids", "attention_mask"])
encoded_dataset_val

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 116
})

In [25]:
val_training_args = TrainingArguments(
    output_dir="./results",
    # per_device_eval_batch_size=8,
    # num_train_epochs=80,
    # weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)
val_trainer = Trainer(
    model=modified_model,  # model is your trained model
    args=val_training_args,
    tokenizer=tokenizer
)

# Run predictions
predictions = val_trainer.predict(encoded_dataset_val)
pred_logits = predictions.predictions

<ipython-input-25-f70e472841b4>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  val_trainer = Trainer(


In [26]:
pred_labels = (torch.sigmoid(torch.tensor(pred_logits)) > 0.5).int()

In [27]:
pred_labels

tensor([[0, 1, 0, 0, 1],
        [0, 1, 0, 1, 1],
        [0, 0, 0, 0, 1],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [1, 1, 0, 0, 1],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 1, 0, 1, 1],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],


In [28]:
df_val = pd.DataFrame(pred_labels, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])
df_val

,Anger,Fear,Joy,Sadness,Surprise
0,0,1,0,0,1
1,0,1,0,1,1
2,0,0,0,0,1
3,0,1,1,1,0
4,0,0,0,0,0
...,...,...,...,...,...
111,0,1,0,0,1
112,0,0,0,0,1
113,0,1,0,1,0
114,0,1,0,1,0


In [29]:
df_val.insert(loc=0, column='id', value='text')
df_val['id'] = [f'eng_dev_track_a_{i+1:05}' for i in range(len(df_val))]
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,1,0,0,1
1,eng_dev_track_a_00002,0,1,0,1,1
2,eng_dev_track_a_00003,0,0,0,0,1
3,eng_dev_track_a_00004,0,1,1,1,0
4,eng_dev_track_a_00005,0,0,0,0,0
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,0,0,1
112,eng_dev_track_a_00113,0,0,0,0,1
113,eng_dev_track_a_00114,0,1,0,1,0
114,eng_dev_track_a_00115,0,1,0,1,0


In [30]:
df_val.to_csv('AUGMENTED_my_pred_after_FCLayer_and_lastlayer_twitterRoberta.csv', index=False)